<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:
What recieves input from our data set. Generally has one node per feature.
### Hidden Layer:
Magic in between layers that we don't really care what they output since it gets piped in between our input and output layers.
### Output Layer:
Prediction of target.
### Neuron:
A single perceptron, takes in X inputs outputs one output, can be chained together into neural nets.
### Weight:
Scalar for inputs.
### Activation Function:
Binary classifier.
### Node Map:
How neurons, inputs, outputs are connected.
### Perceptron:
Algorithm for supervised learning of binary classifiers.

## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

#### Your Answer Here

## Write your own perceptron code that can correctly classify (99.0% accuracy) a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [2]:
import pandas as pd
data = { 'x1': [0,1,0,1],
         'x2': [0,0,1,1],
         'y':  [1,1,1,0]
       }

df = pd.DataFrame.from_dict(data).astype('int')

In [3]:
X = df.loc[:,['x1','x2']].values
X

array([[0, 0],
       [1, 0],
       [0, 1],
       [1, 1]])

In [4]:
def sigmoid(x):
    return 1 / (1+np.exp(-x))

# used to update weights
def sigmoid_derivate(x):
    sx = sigmoid(x)
    return sx * (1 - sx)

In [5]:
import numpy as np

weights = 2 * np.random.random((2,1)) - 1
weights

array([[ 0.79835231],
       [-0.22936778]])

In [6]:
weighted_sum = np.dot(X, weights)
weighted_sum

array([[ 0.        ],
       [ 0.79835231],
       [-0.22936778],
       [ 0.56898453]])

In [7]:
activated_output = sigmoid(weighted_sum)
activated_output

array([[0.5       ],
       [0.68962191],
       [0.44290813],
       [0.63852883]])

In [8]:
correct = df['y'].values
correct = [[i] for i in correct]
correct

[[1], [1], [1], [0]]

In [9]:
error = correct - activated_output
error

array([[ 0.5       ],
       [ 0.31037809],
       [ 0.55709187],
       [-0.63852883]])

In [11]:
adjustments = error * sigmoid_derivate(activated_output)
adjustments

array([[ 0.11750186],
       [ 0.06905381],
       [ 0.13266002],
       [-0.14440598]])

In [12]:
weights += np.dot(X.T, adjustments)

In [13]:
for i in range(10000):
    weighted_sum = np.dot(X, weights)
    activated_output = sigmoid(weighted_sum)
    error = correct - activated_output
    adjustments = error * sigmoid_derivate(activated_output)
    weights += np.dot(X.T, adjustments)
    
print("----------")
print("Weights after training")
print(weights)
print("----------")

print("----------")
print("Output after training")
print(activated_output)
print("----------")

----------
Weights after training
[[ 1.52655666e-16]
 [-2.91433544e-16]]
----------
----------
Output after training
[[0.5]
 [0.5]
 [0.5]
 [0.5]]
----------


## Implement your own Perceptron Class and use it to classify a binary dataset: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [15]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Although neural networks can handle non-normalized data, scaling or normalizing your data will improve your neural network's learning speed. Try to apply the sklearn `MinMaxScaler` or `Normalizer` to your diabetes dataset. 

In [23]:
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

feats = list(diabetes)[:-1]

X = StandardScaler().fit_transform(diabetes[feats].values)

y = diabetes["Outcome"]

In [24]:
class Perceptron:
    
    def __init__(self, niter = 10):
        self.__niter = niter
        
    
    def __sigmoid(self, x):
        return 1 / (1. + np.exp(-x))
    
    
    def __sigmoid_derivative(self, x):
        sx = self.__sigmoid(x)
        return sx * (1-sx)
    

    def fit(self, X, y):
        """
            Fit training data
            X : Training vectors, X.shape : [#samples, #features]
            y : Target values, y.shape : [#samples]
        """
        # Randomly Initialize Weights
        self.__weights = np.random.random(X.shape[1])

        for i in range(self.__niter):
            
            # Weighted sum of inputs / weights
            weighted_sum = np.dot(X, self.__weights)
            
            # Activate!
            activated_output = self.__sigmoid(weighted_sum)

            # Calc error
            error = y - activated_output
            adjustments = error * self.__sigmoid_derivative(activated_output)
            
            # Update the Weights
            self.__weights += np.dot(X.T, adjustments)
        
        return self
    

    def predict(self, X):
        """Return class label after unit step"""
        weighted_sum = np.dot(X, self.__weights)
        return self.__sigmoid(weighted_sum)

In [25]:
perc = Perceptron(1000)
perc.fit(X, y)

In [27]:
from sklearn.metrics import accuracy_score

pred = perc.predict(X).astype(np.int64)
score = 100*accuracy_score(pred, y)
print(f"Accuracy: {score:,.2f}%")

Accuracy: 70.05%


## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?